## Question 1

@Sparsha-Sherke ➜ /workspaces/llm_zoomcamp (main) $ curl localhost:9200
{
  "name" : "49438d26e4b3",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "5WummTRsRSKf72HbzpIBjg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}

## Question 2

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200') 

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework-sparsha"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework-sparsha'})

In [5]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████| 948/948 [00:27<00:00, 35.03it/s]


## Question 3

In [7]:
query = 'How do I execute a command in a running docker container?'

In [8]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "document": hit['_source'],
            "score": hit['_score']
        })
    
    return result_docs

def find_top_score(query):
    results = elastic_search(query)
    if results:
        top_score = results[0]["score"]
        return top_score
    else:
        return None

# Example query
query = "How do I execute a command in a running docker container?"
top_score = find_top_score(query)

print(f"Top score for the query: {top_score}")

Top score for the query: 75.54128


Output of above cell : "Top score for the query: 75.54128". Hence, the closest option 74.05 was chosen.

## Question 4

In [9]:
def search(query, course):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "question": hit['_source']['question'],
            "score": hit['_score']
        })
    
    if len(result_docs) >= 3:
        return result_docs[2]["question"]
    else:
        return None

query = "How do I execute a command in a running docker container?"
course = "machine-learning-zoomcamp"
third_question = search(query, course)

print(f"The 3rd question returned by the search engine: {third_question}")

The 3rd question returned by the search engine: How do I copy files from a different folder into docker container’s working directory?


## Question 5

In [10]:
def search(query, course):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "question": hit['_source']['question'],
            "text": hit['_source']['text']
        })
    
    context = ""
    for doc in result_docs:
        context += context_template.format(question=doc["question"], text=doc["text"]) + "\n\n"
    
    return context

query = "How do I execute a command in a running docker container?"
course = "machine-learning-zoomcamp"
context_template = """
Q: {question}
A: {text}
""".strip()

In [11]:
context = search(query, course)

print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [12]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

question = "How do I execute a command in a running docker container?"
context = search(query, course)

prompt = prompt_template.format(question=question, context=context)

print(prompt)

print("The length of the resulting prompt is:", len(prompt))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

## Question 6

In [15]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [13]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [14]:
print(encoding.encode(prompt))

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480, 734, 1385, 51

In [15]:
print(len(encoding.encode(prompt)))

323


## Ungraded 

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [20]:
llm(prompt)

' To execute a command in a running docker container, you can use the `docker exec` command followed by `-it <container-id> bash`. First, identify the specific container ID using `docker ps`, then run the following command:\n\n```bash\ndocker exec -it <container-id> <command_to_execute>\n```'